In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch_geometric.transforms as T
from torch import lgamma
from torch_geometric.data import DataLoader
from torch_scatter import scatter_mean
import argparse
import numpy as np
import random
import ogb
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator

parser = argparse.ArgumentParser(description='PyTorch implementation of relative positional encodings for graph Transformers')
args = parser.parse_args("")
args.device = 3
args.device = torch.device('cuda:'+ str(args.device) if torch.cuda.is_available() else 'cpu')
print("device:", args.device)
torch.manual_seed(0)
np.random.seed(0)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(0)

def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed = 0
set_seed(seed)

device: cuda:3


In [2]:
class MultiHeadedAttention_RPR(nn.Module):
    def __init__(self, d_model, h, vocab_size=2, dropout=.0):
        """
        multi-head attention
        :param h: nhead
        :param d_model: d_model
        :param dropout: float
        """
        super(MultiHeadedAttention_RPR, self).__init__()
        assert d_model % h == 0
        #  assume d_v always equals d_k
        
        self.d_k = d_model // h
        self.h = h
        self.linears = nn.ModuleList([nn.Linear(d_model, d_model) for i in range(4)])
        self.dropout = nn.Dropout(p=dropout)

        self.vocab_size = vocab_size
        self.embed_K = nn.Embedding(self.vocab_size, self.d_k)
        self.embed_V = nn.Embedding(self.vocab_size, self.d_k)

    def forward(self, query, key, value, rpr_matrix, mask=None):
        nbatches = query.size(0)  # batch size
        seq_len = query.size(1)
        # 1) split embedding dim to h heads : from d_model => h * d_k
        # dim: (nbatch, h, seq_length, d_model//h)
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]

        # 2) rpr
        assert rpr_matrix.size(1) == seq_len and rpr_matrix.size(2) == seq_len
        relation_keys = self.embed_K(rpr_matrix)
        relation_values = self.embed_V(rpr_matrix)
        logits = self._relative_attn_inner(query, key, relation_keys, True)
        weights = self.dropout(F.softmax(logits, -1))
        x = self._relative_attn_inner(weights, value, relation_values, False)
        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

    def _relative_attn_inner(self, x, y, z, transpose):
        nbatches = x.size(0)
        heads = x.size(1)
        seq_len = x.size(2)

        # (N, h, s, d) @ (N, h, d, s) => (N, h, s, s)
        # (N, h, s, s) @ (N, h, s, d) => (N, h, s, d)
        xy_matmul = torch.matmul(x, y.transpose(-1, -2) if transpose else y)
        # (s, N, h, d)
        # (s, N, h, s)
        x_t_v = x.permute(2, 0, 1, 3).contiguous()
        # (s, N, h, d) @ (s, N, d, s) => (s, N, h, s)
        # (s, N, h, s) @ (s, N, s, d) => (s, N, h, d)
        x_tz_matmul = torch.matmul(x_t_v, z.permute(1, 0, 3, 2) if transpose else z.transpose(0, 1))
        # (N, h, s, s)
        # (N, h, s, d)
        x_tz_matmul_v_t = x_tz_matmul.permute(1, 2, 0, 3)
        return xy_matmul + x_tz_matmul_v_t

In [3]:
import torch
import copy
from torch import Tensor
import torch.nn.functional as F
from torch.nn import Module
from torch.nn import ModuleList
from torch.nn.init import xavier_uniform_
from torch.nn import Dropout
from torch.nn import Linear
from torch.nn import LayerNorm

def _get_clones(module, N):
    return ModuleList([copy.deepcopy(module) for i in range(N)])

def _get_activation_fn(activation):
    if activation == "relu":
        return F.relu
    elif activation == "gelu":
        return F.gelu

    raise RuntimeError("activation should be relu/gelu, not {}".format(activation))

class TransformerEncoderLayer(Module):
    r"""TransformerEncoderLayer is made up of self-attn and feedforward network.
    This standard encoder layer is based on the paper "Attention Is All You Need".
    Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N Gomez,
    Lukasz Kaiser, and Illia Polosukhin. 2017. Attention is all you need. In Advances in
    Neural Information Processing Systems, pages 6000-6010. Users may modify or implement
    in a different way during application.

    Args:
        d_model: the number of expected features in the input (required).
        nhead: the number of heads in the multiheadattention models (required).
        dim_feedforward: the dimension of the feedforward network model (default=2048).
        dropout: the dropout value (default=0.1).
        activation: the activation function of intermediate layer, relu or gelu (default=relu).

    Examples::
        >>> encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
        >>> src = torch.rand(10, 32, 512)
        >>> out = encoder_layer(src)
    """

    def __init__(self, d_model, nhead, dim_feedforward=2048, dropout=0.0, activation="relu"):
        super(TransformerEncoderLayer, self).__init__()
        self.self_attn = MultiHeadedAttention_RPR(d_model, nhead, 2, dropout=dropout)
        # Implementation of Feedforward model
        self.linear1 = Linear(d_model, dim_feedforward)
        self.dropout = Dropout(dropout)
        self.linear2 = Linear(dim_feedforward, d_model)

        self.norm1 = LayerNorm(d_model)
        self.norm2 = LayerNorm(d_model)
        self.dropout1 = Dropout(dropout)
        self.dropout2 = Dropout(dropout)

        self.activation = _get_activation_fn(activation)

    def __setstate__(self, state):
        if 'activation' not in state:
            state['activation'] = F.relu
        super(TransformerEncoderLayer, self).__setstate__(state)

    def forward(self, src, rpr_matrix, src_mask):
        r"""Pass the input through the encoder layer.

        Args:
            src: the sequence to the encoder layer (required).
            src_mask: the mask for the src sequence (optional).
        Shape:
            see the docs in Transformer class.
        """
        src2 = self.self_attn(src, src, src, rpr_matrix, mask=src_mask)[0]
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        src2 = self.linear2(self.dropout(self.activation(self.linear1(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src

class TransformerEncoder(Module):
    r"""TransformerEncoder is a stack of N encoder layers

    Args:
        encoder_layer: an instance of the TransformerEncoderLayer() class (required).
        num_layers: the number of sub-encoder-layers in the encoder (required).
        norm: the layer normalization component (optional).

    Examples::
        >>> encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)
        >>> transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)
        >>> src = torch.rand(10, 32, 512)
        >>> out = transformer_encoder(src)
    """
    __constants__ = ['norm']

    def __init__(self, encoder_layer, num_layers, norm=None):
        super(TransformerEncoder, self).__init__()
        self.layers = _get_clones(encoder_layer, num_layers)
        self.num_layers = num_layers
        self.norm = norm

    def forward(self, src, rpr_matrix, mask=None):
        r"""Pass the input through the encoder layers in turn.

        Args:
            src: the sequence to the encoder (required).
            mask: the mask for the src sequence (optional).
            src_key_padding_mask: the mask for the src keys per batch (optional).

        Shape:
            see the docs in Transformer class.
        """
        output = src

        for mod in self.layers:
            output = mod(output, rpr_matrix, src_mask=mask)

        if self.norm is not None:
            output = self.norm(output)

        return output

In [4]:
args.dataset = 'ogbg-moltox21'
args.n_classes = 12
args.batch_size = 128
args.lr = 0.001
args.graph_pooling = 'mean'
args.proj_mode = 'nonlinear'
args.eval_metric = 'rocauc'

In [5]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from ogb.graphproppred.mol_encoder import AtomEncoder
from torch_geometric.nn import global_add_pool, global_mean_pool, global_max_pool
from torch_geometric.utils import to_dense_adj, to_dense_batch

class TransformerModel(nn.Module):

    def __init__(self, nclasses, ninp, nhead, nhid, nlayers, dropout=0.0):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        # self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = AtomEncoder(emb_dim=ninp)
        self.ninp = ninp
        
        #Different kind of graph pooling
        if args.graph_pooling == "sum":
            self.graph_pool = global_add_pool
        elif args.graph_pooling == "mean":
            self.graph_pool = global_mean_pool
        elif args.graph_pooling == "max":
            self.graph_pool = global_max_pool
        else:
            raise ValueError("Invalid graph pooling type.")
              
        #One more MLP to project the final embedding
        if args.proj_mode == 'linear':
            self.projector = nn.Linear(nhid, nhid, bias=False)
        elif args.proj_mode == 'nonlinear':
            self.projector = nn.Sequential(
                nn.Linear(nhid, nhid),
                nn.BatchNorm1d(nhid),
                nn.ReLU(),
                nn.Linear(nhid, nhid, bias=False)
            )
        
        self.task_pred = nn.Sequential(
            nn.Linear(nhid, nhid),
            nn.BatchNorm1d(nhid),
            nn.ReLU(),
            nn.Linear(nhid, nclasses)
        )

    # is mask needed?
    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, src, src_mask=None): 
#         data_list = src.to_data_list()
#         seq_len = torch.max(torch.unique(src.batch, return_counts=True)[1])
    
#         raw_features = torch.stack([F.pad(input=self.encoder(d.x), pad=(0, 0, 0, seq_len - d.x.size(0))) for d in data_list])
#         dense_adj_list = [to_dense_adj(d.edge_index)[0] for d in data_list]
#         rpr_matrix = torch.stack([F.pad(input=d, pad=(0, seq_len - d.size(1), 0, seq_len - d.size(0))) for d in dense_adj_list]).long()
#         print(raw_features.size(), rpr_matrix.size())
        
        raw_features, mask = to_dense_batch(self.encoder(src.x), batch=src.batch, fill_value=0)
        rpr_matrix = to_dense_adj(src.edge_index, batch=src.batch, max_num_nodes=raw_features.size(1)).long()
#         print(raw_features.size(), rpr_matrix.size())
        
        output = self.transformer_encoder(raw_features, rpr_matrix, src_mask)
        
        graph_emb = self.graph_pool(output[mask], src.batch)   
        graph_pred = self.task_pred(graph_emb)
        
        return graph_pred

In [6]:
print("Loading data...")
print("dataset: {} ".format(args.dataset))
dataset = PygGraphPropPredDataset(name=args.dataset).shuffle()

# loader = DataLoader(dataset, batch_size=len(dataset), shuffle=False, drop_last=True)

# for idx, batch in enumerate(loader):
#     data_list = batch.to_data_list()
#     seq_len = max([d.x.size(0) for d in data_list])
# print(seq_len)

Loading data...
dataset: ogbg-moltox21 


In [7]:
train_loader = DataLoader(dataset[:int(0.8 * len(dataset))], batch_size=args.batch_size, shuffle=False, drop_last=True)
test_loader = DataLoader(dataset[int(0.8 * len(dataset)):], batch_size=args.batch_size, shuffle=False, drop_last=True)

emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.0 # the dropout value
model = TransformerModel(args.n_classes, emsize, nhead, nhid, nlayers, dropout).to(args.device)

# To train Transformer, better to use the default hyperparameter for optimizer and LR scheduler.
# You can refer to some latest code as a reference.
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
criterion = torch.nn.BCEWithLogitsLoss(reduction = "mean")
evaluator = Evaluator(name=args.dataset)

args.num_epochs = 50
for epoch in range(args.num_epochs):
    ############
    # TRAINING #
    ############
    
    model.train()
    
    loss_epoch = 0
    for idx, batch in enumerate(train_loader):
        z = model(batch.to(args.device))

        y = batch.y.float()
        is_valid = ~torch.isnan(y)

        optimizer.zero_grad()
        loss = criterion(z[is_valid], y[is_valid])
        loss.backward()
        optimizer.step()

        loss_epoch += loss.item()

    print('Train loss:', loss_epoch / len(train_loader))
    
    ##############
    # EVALUATION #
    ##############
    
    model.eval()
    
    with torch.no_grad():
        loss_epoch = 0
        y_true = []
        y_scores = []
        for idx, batch in enumerate(test_loader):
            z = model(batch.to(args.device))

            y = batch.y.float()
            y_true.append(y)
            y_scores.append(z)
            is_valid = ~torch.isnan(y)

            optimizer.zero_grad()
            loss = criterion(z[is_valid], y[is_valid])

            loss_epoch += loss.item()
        
        y_true = torch.cat(y_true, dim = 0)
        y_scores = torch.cat(y_scores, dim = 0)

    input_dict = {"y_true": y_true, "y_pred": y_scores}
    result_dict = evaluator.eval(input_dict)
    print('Test loss:', loss_epoch / len(test_loader))
    print('Test ROC-AUC:', result_dict[args.eval_metric])

Train loss: 0.3840306888644894
Test loss: 0.47244936724503833
Test ROC-AUC: 0.6078850311304983
Train loss: 0.24866258477171263
Test loss: 0.45485736678044003
Test ROC-AUC: 0.6278808713535898
Train loss: 0.23447011100749174
Test loss: 0.5805706158280373
Test ROC-AUC: 0.6404235296482289
Train loss: 0.22544364413867393
Test loss: 0.6687075396378835
Test ROC-AUC: 0.5747901133884529
Train loss: 0.21939901914447546
Test loss: 0.32966379697124165
Test ROC-AUC: 0.6951001083494525
Train loss: 0.21761066373437643
Test loss: 1.5294218858083088
Test ROC-AUC: 0.5897350719471978
Train loss: 0.21375601335118213
Test loss: 0.7790202548106512
Test ROC-AUC: 0.560669036097115
Train loss: 0.21132949894915024
Test loss: 0.524609386920929
Test ROC-AUC: 0.6774632435590905
Train loss: 0.2137811646486322
Test loss: 0.49296770989894867
Test ROC-AUC: 0.6868637310273492
Train loss: 0.21225193267067274
Test loss: 0.7828775122761726
Test ROC-AUC: 0.5962532311141459
Train loss: 0.20729540505756935
Test loss: 0.92776